In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!apt-get install p7zip 
!apt-get install p7zip-full 

In [ ]:
!ls

In [ ]:
import csv
import pathlib


filename = 'submit.csv'
p_tmp = pathlib.Path('../input/test-img/test').glob('*.png')
with open(filename, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id','label'])
for p in p_tmp:
    with open(filename, 'a', newline="")as f:
        writer = csv.writer(f)
        writer.writerow([p.name,"cat"])
        


In [ ]:
!7za e ../input/cifar-10/train.7z

In [ ]:
!pip install natsort

In [ ]:
from tensorflow.keras.preprocessing.image import  load_img,img_to_array,array_to_img
from glob import glob
import csv
from natsort import natsorted

CATEGORYS = ['airplane','automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

train_img = []
train_label = []
files = glob("./*.png")

csv_file = open("../input/cifar-10/trainLabels.csv", "r", encoding="utf_8", errors="", newline="")
label_read = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
label_list = []
for row in label_read:
    label_list.append(row)

print(label_list[1])
i = 0
for file_name in natsorted(files):
    # print(file_name)
    img = load_img(file_name,color_mode="rgb", target_size=(32,32))
    array = img_to_array(img)
    train_img.append(array)
    categorys_num = CATEGORYS.index(label_list[i+1][1])
    train_label.append(categorys_num)
    i += 1



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
train_img = np.array(train_img)
train_label = np.array(train_label)
train_img = train_img.astype('float32') / 255
train_label = to_categorical(train_label,10)


for i in range(len(train_img)):
    plt.imshow(train_img[i])
    print(CATEGORYS[np.argmax(train_label[i])])
    plt.show()
    if i == 9:
        break

In [ ]:
from tensorflow.keras.datasets import cifar10

(a,b), (val_img, val_label) = cifar10.load_data()
val_img = val_img.astype("float32")/255
val_label = to_categorical(val_label, 10)

In [ ]:
for i in range(11,21):
    plt.imshow(val_img[i])
    print("label", CATEGORYS[np.argmax(val_label[i])])
    plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    featurewise_center = False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(
    # featurewise_center = False,
    # featurewise_std_normalization=False,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True
)

# datagen.fit(train_img,augment=True, rounds=2, seed=None)


train_img_augment = train_datagen.flow(train_img, train_label,batch_size=128)
val_img_augment = val_datagen.flow(val_img, val_label,batch_size=128)

In [ ]:
def step_decay(epoch):
    lr = 0.001
    if(epoch >= 100):
        lr/=5
    if(epoch>=140):
        lr/=2
    return lr

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense,MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import LearningRateScheduler

model = Sequential()
model.add(Conv2D(64, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPool2D())
model.add(Conv2D(128, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(0.25))
model.add(MaxPool2D())
# model.add(Conv2D(256, (3,3), padding='valid'))
# model.add(Activation('relu'))
model.add(Conv2D(256, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(10))
model.add(Activation('softmax'))

lr_decay = LearningRateScheduler(step_decay)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001),metrics=['accuracy'])
# input_shape = train_img[0].shape
# print(input_shape)
# model.summary()
# history = model.fit(train_img, train_label, batch_size=16, epochs=150,verbose=1,validation_data=(val_img,val_label))
histroy = model.fit_generator(train_img_augment,epochs=150,validation_data=val_img_augment,callbacks=[lr_decay])

In [ ]:
model.summary()

In [ ]:
plt.plot(histroy.history["accuracy"])
plt.plot(histroy.history["val_accuracy"])
plt.title("Model accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(histroy.history["loss"])
plt.plot(histroy.history["val_loss"])
plt.title("model loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
!ls

In [ ]:
import csv
import pathlib


filename = 'submit.csv'
p_tmp = pathlib.Path('../input/test-img/test').glob('*.png')
with open(filename, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id','label'])
cnt = 0
for p in p_tmp:
    
    img = load_img(p,color_mode="rgb", target_size=(32,32))
    array = img_to_array(img)
    np_array = np.array(array)
    test_img = np_array.astype('float32') / 255
    test_img = test_img.reshape(-1,32, 32, 3)
    # print(test_img.shape)
    predictions = model.predict(test_img)
    predict_label = CATEGORYS[predictions[0].argmax()]
    # print(predict_label)
    
    with open(filename, 'a', newline="")as f:
        writer = csv.writer(f)
        writer.writerow([p.stem,predict_label])
    cnt += 1
    if cnt % 10000 == 0:
        print('cnt',cnt)
    
        
    

        
        
'''   
files = glob("./*.png")

csv_file = open("../input/cifar-10/trainLabels.csv", "r", encoding="utf_8", errors="", newline="")
label_read = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
label_list = []
for row in label_read:
    label_list.append(row)

print(label_list[1])
i = 0
for file_name in natsorted(files):
    # print(file_name)
    img = load_img(file_name,color_mode="rgb", target_size=(32,32))
    array = img_to_array(img)
    train_img.append(array)
    categorys_num = CATEGORYS.index(label_list[i+1][1])
    train_label.append(categorys_num)
    i += 1
'''    
